<a href="https://colab.research.google.com/github/rakib89-ftj/ml--models/blob/master/Traffic_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('/content/Traffic-Prediction-using-Machine-Learning.csv')
df.head()

,Day,Date,CodedDay,Zone,Weather,Temperature,Traffic
0,Wednesday,01-06-18,3,2,35,17,2
1,Wednesday,01-06-18,3,3,36,16,3
2,Wednesday,01-06-18,3,4,27,25,5
3,Wednesday,01-06-18,3,5,23,23,3
4,Wednesday,01-06-18,3,6,18,42,2


In [3]:
df.describe()

,CodedDay,Zone,Weather,Temperature,Traffic
count,1439.000000,1439.000000,1439.000000,1439.000000,1439.000000
mean,4.000695,72.549687,23.886032,25.011119,3.031967
std,1.733054,41.554330,13.576787,11.325376,1.429018
min,1.000000,1.000000,0.000000,6.000000,1.000000
25%,3.000000,37.000000,12.000000,15.000000,2.000000
50%,4.000000,73.000000,24.000000,25.000000,3.000000
75%,5.000000,108.500000,35.000000,35.000000,4.000000
max,7.000000,144.000000,47.000000,45.000000,5.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1439 entries, 0 to 1438
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Day          1439 non-null   object
 1   Date         1439 non-null   object
 2   CodedDay     1439 non-null   int64 
 3   Zone         1439 non-null   int64 
 4   Weather      1439 non-null   int64 
 5   Temperature  1439 non-null   int64 
 6   Traffic      1439 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 78.8+ KB


In [5]:
df.isnull().sum()

,0
Day,0
Date,0
CodedDay,0
Zone,0
Weather,0
Temperature,0
Traffic,0


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder

# **Convert 'Day' into one-hot encoding**

In [7]:
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_day = encoder.fit_transform(df[['Day']])

# **Combine one-hot encoded columns with the dataset**

In [8]:
encoded_day_df=pd.DataFrame(encoded_day, columns=encoder.get_feature_names_out(['Day']))
df = pd.concat([df.drop('Day', axis=1), encoded_day_df], axis=1)

In [9]:
df.head()

,Date,CodedDay,Zone,Weather,Temperature,Traffic,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday
0,01-06-18,3,2,35,17,2,0.0,0.0,0.0,0.0,0.0,1.0
1,01-06-18,3,3,36,16,3,0.0,0.0,0.0,0.0,0.0,1.0
2,01-06-18,3,4,27,25,5,0.0,0.0,0.0,0.0,0.0,1.0
3,01-06-18,3,5,23,23,3,0.0,0.0,0.0,0.0,0.0,1.0
4,01-06-18,3,6,18,42,2,0.0,0.0,0.0,0.0,0.0,1.0


# **Features and target**

In [10]:
df.columns

Index(['Date', 'CodedDay', 'Zone', 'Weather', 'Temperature', 'Traffic',
       'Day_Monday', 'Day_Saturday', 'Day_Sunday', 'Day_Thursday',
       'Day_Tuesday', 'Day_Wednesday'],
      dtype='object')

In [11]:
X=df[['Weather', 'Zone', 'Temperature']+ list(encoded_day_df.columns)]
y=df['Traffic']

In [12]:
X.head()

,Weather,Zone,Temperature,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday
0,35,2,17,0.0,0.0,0.0,0.0,0.0,1.0
1,36,3,16,0.0,0.0,0.0,0.0,0.0,1.0
2,27,4,25,0.0,0.0,0.0,0.0,0.0,1.0
3,23,5,23,0.0,0.0,0.0,0.0,0.0,1.0
4,18,6,42,0.0,0.0,0.0,0.0,0.0,1.0


# **Creating Model**

In [13]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)

# **Linear Regression Model**

In [14]:
linear_model=LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_linear= linear_model.predict(X_test)

In [15]:
print('Mean Absolute Error: ', mean_absolute_error(y_test, y_pred_linear))
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred_linear))
print('Root Mean Squared Error: ', np.sqrt(mean_squared_error(y_test, y_pred_linear)))

Mean Absolute Error:  1.2899165871761982
Mean Squared Error:  2.175848274483995
Root Mean Squared Error:  1.4750756843240265


# **Random Forest Model**

In [16]:
forest_model= RandomForestRegressor()
forest_model.fit(X_train, y_train)
y_pred= forest_model.predict(X_test)

In [17]:
print('Mean Absolute Error: ', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error: ', np.sqrt(mean_squared_error(y_test, y_pred)))

Mean Absolute Error:  1.2818055555555554
Mean Squared Error:  2.1991513888888887
Root Mean Squared Error:  1.482953603080315


# **Gradient Boost Regressor**

In [18]:
from sklearn.ensemble import GradientBoostingRegressor
gbr_model= GradientBoostingRegressor()
gbr_model.fit(X_train, y_train)
y_pred_gbr= gbr_model.predict(X_test)


In [19]:
print('Mean Absolute Error: ', mean_absolute_error(y_test, y_pred_gbr))
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred_gbr))
print('Root Mean Squared Error: ', np.sqrt(mean_squared_error(y_test, y_pred_gbr)))

Mean Absolute Error:  1.292386477534089
Mean Squared Error:  2.166539096797232
Root Mean Squared Error:  1.4719168104200835


# **Cross Validation **

In [20]:
from sklearn.model_selection import cross_val_score

scores= cross_val_score(linear_model, X, y, cv=5)
print('Cross Validation Scores: ', scores)
print('Mean Cross Validation Score: ', scores.mean())

Cross Validation Scores:  [-0.01635927 -0.02972785 -0.38983882 -1.04691031 -0.01911775]
Mean Cross Validation Score:  -0.3003908004435731
